## pack_padded_sequence
- https://pytorch.org/docs/1.0.1/nn.html#torch.nn.utils.rnn.pack_padded_sequence
- 在使用深度学习特别是LSTM进行文本分析时，经常会遇到文本长度不一样的情况，此时就需要对同一个batch中的不同文本使用padding的方式进行文本长度对齐，方便将训练数据输入到LSTM模型进行训练，同时为了保证模型训练的精度，应该同时告诉LSTM相关padding的情况，此时，pytorch中的pack_padded_sequence就有了用武之地。
- 通常pading的位置向量都是0，我们需要使用pack_padded_sequence() 把数据压紧，即去掉pading的部分，减少冗余。然后再输入网络中，如lstm等。通过网络后的结果也是压紧的，需要通过pad_packed_sequence()还原。
- torch.nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=False)
```
input (Tensor) – padded batch of variable length sequences.
lengths (Tensor) – list of sequences lengths of each batch element.
batch_first (bool, optional) – if True, the input is expected in B x T x * format.
```
- pad_packed_sequence 解压

In [1]:
import torch
import torch.nn as nn
import torch.nn.utils as utils
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [2]:
print(torch.__version__)

1.0.1


In [46]:
# 定义一个双向lstm网络层
lstm = nn.LSTM(4, 100, num_layers=1, batch_first=True, bidirectional=True)  

In [47]:
# 定义一个有padding的序列数据，也就是有冗余的0
x = torch.tensor([[[1,2,3,4],
                   [2,3,4,5],
                   [2,5,6,0]],
                  [[1,2,1,1],
                   [1,6,7,9],
                   [0,0,0,0]],
                  [[1,2,3,4],
                   [1,1,1,1],
                   [0,0,0,0]],
                  [[1,2,3,4],
                   [0,0,0,0],
                   [0,0,0,0]],
                 ])
x = x.float()
x.size()

torch.Size([4, 3, 4])

In [48]:
# 压紧数据，去掉冗余
packed = pack_padded_sequence(x, torch.tensor([3, 2, 2,1]), batch_first=True)   # 打包，压缩
packed

PackedSequence(data=tensor([[1., 2., 3., 4.],
        [1., 2., 1., 1.],
        [1., 2., 3., 4.],
        [1., 2., 3., 4.],
        [2., 3., 4., 5.],
        [1., 6., 7., 9.],
        [1., 1., 1., 1.],
        [2., 5., 6., 0.]]), batch_sizes=tensor([4, 3, 1]))

In [49]:
# 通过lstm进行计算
output, hidden = lstm(packed)
# 得到的结果也是压紧的
output

PackedSequence(data=tensor([[-0.1277, -0.0269, -0.0070,  ..., -0.0997,  0.0827,  0.0156],
        [-0.0272, -0.0503, -0.0239,  ..., -0.0778,  0.0617, -0.0288],
        [-0.1277, -0.0269, -0.0070,  ..., -0.0498,  0.0694,  0.0405],
        ...,
        [-0.2541, -0.0296, -0.0304,  ..., -0.0526,  0.0918,  0.0056],
        [-0.0754, -0.0565, -0.0133,  ..., -0.0139,  0.0202,  0.0313],
        [-0.2212, -0.0844, -0.0038,  ..., -0.0590, -0.0370,  0.0686]],
       grad_fn=<CatBackward>), batch_sizes=tensor([4, 3, 1]))

In [50]:
# 解压
encoder_outputs, lenghts = pad_packed_sequence(output, batch_first=True)   # 解包

In [51]:
encoder_outputs

tensor([[[-0.1277, -0.0269, -0.0070,  ..., -0.0997,  0.0827,  0.0156],
         [-0.1971, -0.0458, -0.0240,  ..., -0.0674,  0.0357,  0.0496],
         [-0.2212, -0.0844, -0.0038,  ..., -0.0590, -0.0370,  0.0686]],

        [[-0.0272, -0.0503, -0.0239,  ..., -0.0778,  0.0617, -0.0288],
         [-0.2541, -0.0296, -0.0304,  ..., -0.0526,  0.0918,  0.0056],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.1277, -0.0269, -0.0070,  ..., -0.0498,  0.0694,  0.0405],
         [-0.0754, -0.0565, -0.0133,  ..., -0.0139,  0.0202,  0.0313],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.1277, -0.0269, -0.0070,  ..., -0.0408,  0.0545,  0.0386],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<TransposeBackward0>)

In [52]:
encoder_outputs.size()   # size: [3, 3, 200] 双向的，所以是200维度

torch.Size([4, 3, 200])

In [53]:
encoder_outputs = encoder_outputs.contiguous()
encoder_outputs.size()

torch.Size([4, 3, 200])

In [54]:
feature = encoder_outputs.view(-1, 200)
feature.size()

torch.Size([12, 200])

In [55]:
line = nn.Linear(200,200, bias=False)
y = line(feature)
y.size()

torch.Size([12, 200])

In [56]:
lenghts

tensor([3, 2, 2, 1])

In [57]:
## 来看下双向lstm的输出
len(hidden) # 双向lstm 结果是2  

2

In [65]:
h, c  = hidden
h.size()

torch.Size([2, 4, 100])

In [66]:
c.size()

torch.Size([2, 4, 100])

In [70]:
inp = h.transpose(0, 1).contiguous().view(-1, 200)  # view就是reshape
inp.size()

torch.Size([4, 200])

In [61]:
# 定义一个单向lstm网络层
lstm2 = nn.LSTM(4, 100, num_layers=1, batch_first=True) 

In [62]:
out, hid = lstm2(x)

In [63]:
len(hid)

2

In [64]:
hid[0].size()

torch.Size([1, 4, 100])

In [20]:
## 有必要了解一下lstm的输出结果